In [ ]:
import tensorflow as tf

class FusedModel(tf.Module):
  def __init__(self):
    self.color_classifier = tf.saved_model.load('/nfs/general/shared/tf_models/color_classifier')
    self.orientation_classifier=tf.saved_model.load('/nfs/general/shared/tf_models/orientation')

  @tf.function(input_signature=[tf.TensorSpec(shape=[480,640,3], dtype=tf.uint8)])
  def compute(self, image):
    image_cropped_for_color = tf.image.convert_image_dtype(tf.image.crop_to_bounding_box(image, 220, 260, 40, 80), tf.float32)[tf.newaxis,:,:,:]
    image_for_position_detection = tf.image.convert_image_dtype(tf.image.rgb_to_grayscale(tf.image.crop_to_bounding_box(image, 220,0,100,640)), tf.float32)[tf.newaxis,:,:,:]
    orientation=self.orientation_classifier(image_for_position_detection)[0,:]
    return tf.math.softmax(self.color_classifier(image_cropped_for_color))[0,:], orientation


model = FusedModel()
tf.saved_model.save(model, '/tmp/adder')

converter = tf.lite.TFLiteConverter.from_saved_model('/tmp/adder') 

tflite_model = converter.convert()
open('/nfs/general/shared/tflite/fused_model.tflite', 'wb').write(tflite_model)



image = tf.io.decode_jpeg(tf.io.read_file('/nfs/general/shared/pos/0/1650270351460_.jpg'))

model.compute(image)